# Week 9 Homework: Serverless Deployment

This homework focuses on deploying the Straight vs Curly Hair Type model (trained in Week 8) as a serverless function using ONNX, Docker, and AWS Lambda.


## Prerequisites

Tools:
- ONNX Runtime
- PIL (Pillow)
- NumPy
- Docker (for later questions)


In [3]:
# # Install required packages (if needed)
!pip install onnxruntime pillow numpy

## Model Download

Download the ONNX model files from the repository. These files contain the pre-trained hair type classifier model.


In [4]:
import os
import urllib.request

# Model download URLs
MODEL_PREFIX = "https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle"
MODEL_DATA_URL = f"{MODEL_PREFIX}/hair_classifier_v1.onnx.data"
MODEL_URL = f"{MODEL_PREFIX}/hair_classifier_v1.onnx"

# Local filenames
MODEL_DATA_FILENAME = "hair_classifier_v1.onnx.data"
MODEL_FILENAME = "hair_classifier_v1.onnx"


def download_file(url, filename):
    """Download a file from URL if it doesn't already exist.

    Args:
        url (str): URL to download from.
        filename (str): Local filename to save to.
    """
    if not os.path.exists(filename):
        print(f"Downloading {filename} from {url}...")
        urllib.request.urlretrieve(url, filename)
        print(f"Download complete: {filename}")
    else:
        print(f"File already exists: {filename}")


# Download model files
download_file(MODEL_DATA_URL, MODEL_DATA_FILENAME)
download_file(MODEL_URL, MODEL_FILENAME)


File already exists: hair_classifier_v1.onnx.data
File already exists: hair_classifier_v1.onnx


## Question 1: Input and Output Node Names

To use the ONNX model, we need to know the names of the input and output nodes. Let's inspect the model to find these names.


### Model Structure Inspection

In [7]:
!pip install onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 99.9 MB/s eta 0:00:00:00:0100:01


In [8]:
import onnx

onnx_model = onnx.load(MODEL_FILENAME)
graph = onnx_model.graph

# Input and output information
print("Inputs:")
for inp in graph.input:
    shape = [dim.dim_value if dim.dim_value > 0 else '?' 
             for dim in inp.type.tensor_type.shape.dim]
    print(f"  {inp.name}: {shape}")

print("\nOutputs:")
for out in graph.output:
    shape = [dim.dim_value if dim.dim_value > 0 else '?' 
             for dim in out.type.tensor_type.shape.dim]
    print(f"  {out.name}: {shape}")

# Store output name
OUTPUT_NODE_NAME = graph.output[0].name
print(f"\nOutput node name: {OUTPUT_NODE_NAME}")

Inputs:
  input: ['?', 3, 200, 200]

Outputs:
  output: ['?', 1]

Output node name: output


### Model Layers


In [9]:
# Display model layers (nodes)
print(f"Total layers: {len(graph.node)}\n")

for i, node in enumerate(graph.node, 1):
    inputs = ', '.join(node.input) if node.input else 'N/A'
    outputs = ', '.join(node.output) if node.output else 'N/A'
    print(f"{i:3d}. {node.op_type:20s} | {inputs} -> {outputs}")


Total layers: 9

  1. Shape                | input -> val_0
  2. Conv                 | input, conv1.weight, conv1.bias -> conv2d
  3. Relu                 | conv2d -> relu
  4. MaxPool              | relu -> max_pool2d
  5. Concat               | val_0, val_4 -> val_5
  6. Reshape              | max_pool2d, val_5 -> view
  7. Gemm                 | view, fc1.weight, fc1.bias -> linear
  8. Relu                 | linear -> relu_1
  9. Gemm                 | relu_1, fc2.weight, fc2.bias -> output


## Question 2: Target Size

Download and resize the test image to the target size used in Week 8 preprocessing (200×200).


In [10]:
from io import BytesIO
from urllib import request
from PIL import Image
import numpy as np

IMAGE_URL = "https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg"
TARGET_SIZE = (200, 200)  # from Week 8 preprocessing


def download_image(url: str) -> Image.Image:
    """Download an image from a URL into a PIL RGB image."""
    with request.urlopen(url) as resp:
        buffer = resp.read()
    return Image.open(BytesIO(buffer)).convert("RGB")


def prepare_image(img: Image.Image, target_size=TARGET_SIZE) -> Image.Image:
    """Convert to RGB and resize to the given target size."""
    if img.mode != "RGB":
        img = img.convert("RGB")
    return img.resize(target_size, Image.NEAREST)


In [11]:
# Download and resize
test_img = download_image(IMAGE_URL)
resized_img = prepare_image(test_img, TARGET_SIZE)

print(f"Original size: {test_img.size}")
print(f"Resized size:  {resized_img.size}")

Original size: (1024, 1024)
Resized size:  (200, 200)


In [12]:
type(resized_img)

PIL.Image.Image

In [13]:
# Convert resized image to NumPy array for the next steps
img_array = np.array(resized_img)
print(f"Array shape: {img_array.shape}")
print(f"First pixel (R, G, B): {img_array[0, 0, :].tolist()}")

Array shape: (200, 200, 3)
First pixel (R, G, B): [61, 104, 22]


## Question 3: Preprocess and First-Pixel Value (R channel)

Apply the same preprocessing as in Week 8 (ToTensor + ImageNet normalization) and report the first pixel's R value after normalization.


In [14]:
IMAGENET_MEAN = np.array([0.485, 0.456, 0.406], dtype=np.float32)
IMAGENET_STD = np.array([0.229, 0.224, 0.225], dtype=np.float32)


def preprocess_to_tensor(img_array: np.ndarray) -> np.ndarray:
    """Convert HWC uint8 image to CHW float32 tensor and apply ImageNet normalization."""
    # ToTensor: scale to [0,1] and reorder to CHW
    tensor = img_array.astype(np.float32) / 255.0
    tensor = np.transpose(tensor, (2, 0, 1))  # HWC -> CHW
    # Normalize per channel
    tensor = (tensor - IMAGENET_MEAN[:, None, None]) / IMAGENET_STD[:, None, None]
    return tensor


# Preprocess and extract first-pixel R value
preprocessed = preprocess_to_tensor(img_array)
first_pixel_r = float(preprocessed[0, 0, 0])

print(f"Preprocessed tensor shape (C,H,W): {preprocessed.shape}")
print(f"First pixel R channel after normalization: {first_pixel_r}")


Preprocessed tensor shape (C,H,W): (3, 200, 200)
First pixel R channel after normalization: -1.0732940435409546


## Question 4: Run the Model on the Image

Run inference with ONNX Runtime on the preprocessed tensor and report the model output.

In [15]:
import onnxruntime as ort

In [16]:
# Prepare batch dimension (N, C, H, W)
input_tensor = np.expand_dims(preprocessed, axis=0)

# Load model and run inference
session = ort.InferenceSession(MODEL_FILENAME)
input_name = session.get_inputs()[0].name
output = session.run(None, {input_name: input_tensor})

# Extract scalar prediction
model_output = float(output[0].squeeze())
print(f"Model output (logit): {model_output}")

Model output (logit): 0.09156641364097595


## Question 5: Base Image Size

Pull the base image `agrigorev/model-2025-hairstyle:v1` and check its size (in the SIZE column of `docker images`).


In [ ]:
# Pull the base image and list its size
!docker pull agrigorev/model-2025-hairstyle:v1
!docker images agrigorev/model-2025-hairstyle:v1

REPOSITORY                       TAG       IMAGE ID       CREATED      SIZE
agrigorev/model-2025-hairstyle   v1        9e43d5a5323f   6 days ago   921MB


## Question 6: Extend the Image, Add Lambda Code, and Score the Image

The base image `agrigorev/model-2025-hairstyle:v1` already contains `hair_classifier_empty.onnx` (same preprocessing as Week 8). Steps:
1. Extend the base image, install required libs, and add the lambda handler code.
2. Run the container locally.
3. Score the provided image and report the model output.


### Dockerfile (homework.dockerfile)


In [10]:
%%writefile homework.dockerfile
FROM agrigorev/model-2025-hairstyle:v1

# Install Python deps needed for the lambda handler
RUN pip install --no-cache-dir pillow numpy onnxruntime

# Copy lambda handler into the image
COPY lambda_function.py /var/task/lambda_function.py

# Entrypoint is defined by the base Lambda image



Writing homework.dockerfile


### Create lambda_function.py

In [11]:
%%writefile lambda_function.py
import json
from io import BytesIO
from urllib import request

import numpy as np
import onnxruntime as ort
from PIL import Image

MODEL_PATH = "hair_classifier_empty.onnx"
IMAGENET_MEAN = np.array([0.485, 0.456, 0.406], dtype=np.float32)
IMAGENET_STD = np.array([0.229, 0.224, 0.225], dtype=np.float32)
TARGET_SIZE = (200, 200)


def download_image(url: str) -> Image.Image:
    """Download an image from URL and ensure RGB mode."""
    with request.urlopen(url) as resp:
        buffer = resp.read()
    return Image.open(BytesIO(buffer)).convert("RGB")


def prepare_image(img: Image.Image, target_size=TARGET_SIZE) -> np.ndarray:
    """Resize, normalize (ImageNet), and return NCHW float32 tensor with batch dim."""
    img = img.resize(target_size, Image.NEAREST)
    arr = np.array(img).astype(np.float32) / 255.0  # HWC, [0,1]
    arr = np.transpose(arr, (2, 0, 1))               # CHW
    arr = (arr - IMAGENET_MEAN[:, None, None]) / IMAGENET_STD[:, None, None]
    return np.expand_dims(arr, axis=0)               # NCHW


def predict(url: str) -> float:
    session = ort.InferenceSession(MODEL_PATH, providers=["CPUExecutionProvider"])
    input_name = session.get_inputs()[0].name
    img = download_image(url)
    tensor = prepare_image(img)
    output = session.run(None, {input_name: tensor})[0]
    return float(output.squeeze())


def lambda_handler(event, context):
    url = event.get("url") if isinstance(event, dict) else None
    if not url:
        return {
            "statusCode": 400,
            "body": json.dumps({"error": "url is required"}),
        }
    pred = predict(url)
    return {
        "statusCode": 200,
        "body": json.dumps({"prediction": pred}),
    }



Writing lambda_function.py


### Build and Run Locally

```bash
# Build the image
docker build -t hairstyle-lambda -f homework.dockerfile .

# Run the container locally
docker run --rm -p 9000:8080 hairstyle-lambda lambda_function.lambda_handler

# Invoke (Lambda format)
curl -X POST "http://localhost:9000/2015-03-31/functions/function/invocations" \
     -d '{"url": "https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg"}'
```



NOTE: When a Lambda container runs locally with the **AWS Lambda Runtime Interface Emulator (RIE)**, it exposes a fixed HTTP endpoint that mimics the real Lambda Invoke API.
Because of this, every local invocation uses the same request pattern:

* **URL:** `http://localhost:9000/2015-03-31/functions/function/invocations`
* **Method:** `POST`
* **Body:** The JSON event you want to pass to the handler

This endpoint is not configurable—it exists to replicate how AWS Lambda receives events.
Posting your JSON payload to this path triggers the handler inside the container exactly as Lambda would in the cloud.
